# Group by merchants and aggregate other columns

In [1]:
from pyspark.sql import SparkSession, functions as F

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "8g")
    .config("spark.driver.memory", "8g")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/25 20:57:56 WARN Utils: Your hostname, LAPTOP-MMSL4U2D resolves to a loopback address: 127.0.1.1; using 172.17.187.27 instead (on interface eth0)
24/09/25 20:57:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/25 20:57:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Load in data after cleaning tags
trans_final_sdf = spark.read.parquet('../data/merged/merged_transactions_with_tags.parquet')

In [3]:
# Group by on merchant abn, week of year, year, merchant name, and fixed_tags
grouped_sdf = trans_final_sdf.groupBy("merchant_abn",
    F.weekofyear("order_datetime").alias("week"), 
    F.year("order_datetime").alias("year"), "merchant_name", \
    "description", 'revenue_band', 'take_rate', 'industry_segment')\
    .agg(  # aggregate the following columns
    F.sum("dollar_value").alias("revenue"),
    F.sum(F.when(F.col("gender") == "Male", 1).otherwise(0)).alias("male_count"),
    F.sum(F.when(F.col("gender") == "Female", 1).otherwise(0)).alias("female_count"),
    F.sum(F.when(F.col("gender") == "Undisclosed", 1).otherwise(0)).alias("undisclosed_count"),
    F.count("order_id").alias("num_trans"),
    F.count_distinct("postcode").alias("num_postcodes"),
    F.count_distinct("user_id").alias("num_unique_customers"),
    F.avg("consumer_fraud_probability").alias("average_consumer_fraud_probability"),
    F.avg("merchant_fraud_probability").alias("average_merchant_fraud_probability"),
    F.sum(F.when(F.col("is_fraud") == "true", 1).otherwise(0)).alias("fraud_count")
)
grouped_sdf = grouped_sdf.orderBy("merchant_abn", "year", "week")

In [4]:
# New column for average transaction value
grouped_sdf=grouped_sdf.withColumn('mean_trans_val', grouped_sdf.revenue/grouped_sdf.num_trans)
grouped_sdf.show(5, truncate=False)

24/09/25 20:58:21 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/09/25 20:58:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 20:58:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 20:58:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 20:58:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 20:58:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 20:58:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 20:58:32 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but re

+------------+----+----+-------------+---------------------------------------------------------------------------------+------------+---------+----------------+-----------------+----------+------------+-----------------+---------+-------------+--------------------+----------------------------------+----------------------------------+-----------+------------------+
|merchant_abn|week|year|merchant_name|description                                                                      |revenue_band|take_rate|industry_segment|revenue          |male_count|female_count|undisclosed_count|num_trans|num_postcodes|num_unique_customers|average_consumer_fraud_probability|average_merchant_fraud_probability|fraud_count|mean_trans_val    |
+------------+----+----+-------------+---------------------------------------------------------------------------------+------------+---------+----------------+-----------------+----------+------------+-----------------+---------+-------------+--------------------+-

In [5]:
# Save as csv to check
grouped_sdf.coalesce(1).write.csv('../data/curated/revenue.csv', header=True)
# Save as parquet to continue with segments analysis
grouped_sdf.write.mode("overwrite").parquet("../data/curated/revenue.parquet")

24/09/25 20:59:25 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 20:59:25 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 20:59:25 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 20:59:25 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 20:59:25 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 20:59:25 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 20:59:25 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 20:59:25 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/09/25 20:59:25 WARN RowBasedKeyValueBatch: Calling spill() on